In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Train

In [ ]:
import pandas as pd


In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
# !gsutil cp -r gs://vietai_public/viT5/data/vietnews .
# !gsutil cp -r gs://vietai_public/viT5/data/wikilingua .


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
df_train = pd.read_csv("data_viet/vietnews_train.csv")

In [ ]:
df_train = df_train[["original", "summary"]]
df_train.rename(columns = {'original':'inputs'}, inplace = True)
df_train.rename(columns = {'summary':'labels'}, inplace = True)



In [ ]:
df_train

,inputs,labels
0,Bộ_trưởng Sitharaman phát_biểu tại buổi gặp_gỡ...,Bộ_trưởng Quốc_phòng Ấn_Độ bày_tỏ mong_muốn Vi...
1,Tổng_thống Pháp Emmanuel_Macron phấn_khích ăn_...,"Sau màn ăn_mừng phấn_khích trên khán_đài , Emm..."
2,Lực_lượng tuần_duyên Nhật_Bản đang nỗ_lực tìm_...,Một hòn đảo của Nhật nằm sát khu_vực tranh_chấ...
3,"Cảnh_sát thành_phố Barcelona , Tây_Ban_Nha đã ...",Tình_báo Mỹ báo với cảnh_sát Tây_Ban_Nha về ng...
4,"Trước_đây , khán_giả thấy rằng nghệ_sĩ Xuân_Hư...",Nghệ_sĩ Xuân_Hương và MC Thanh_Bạch đã từng đư...
...,...,...
105413,"Dù phải di_chuyển và "" cháy "" ở một lịch_trình...","Chiều 16-1 , Chương_trình Mùa_xuân Biển đảo ti..."
105414,Tại cuộc họp_báo Chính_phủ thường_kỳ chiều 4/5...,“ Trách_nhiệm cá_nhân hoặc trách_nhiệm đối_với...
105415,"Ngày hôm_qua 28/2 , Tổng_thống Francois_Hollan...",Một tay súng bắn tỉa thuộc lực_lượng cảnh_sát ...
105416,Do dịp 30-4 và 1-5 rơi vào ngày cuối tuần nên ...,"Vietnam_Airlines , Vietjet , Jetstar sẽ tăng t..."


In [ ]:
dataset = Dataset.from_pandas(df_train)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

Map (num_proc=8):   0%|          | 0/105418 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
#!pip install --upgrade accelerate

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [ ]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=2,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )



In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,12.748600
1000,0.836600
1500,0.662400
2000,0.633300
2500,0.615500
3000,0.613600
3500,0.589000
4000,0.576600
4500,0.572600
5000,0.558600
